# Constructive Cascading Neural Networks Implementations



In [2]:
# import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.autograd import Variable

#### Importing Dataset

In [3]:
colnames = ["fix1", "fix2", "fix3", "fix4", "recog"]

data_train = pd.read_csv("./dataset/fzis_training.csv", header=None, names=colnames)
data_test = pd.read_csv("./dataset/fzis_testing.csv", header=None, names=colnames)

X_train = pd.DataFrame(data_train.iloc[:,:-1])
Y_train = pd.DataFrame(data_train["recog"])

X = Variable(torch.Tensor(X_train.to_numpy()).float())
Y = Variable(torch.Tensor(Y_train.to_numpy()).long())


### Input and Output definition

In [4]:
dim_in = 4 #input dimension
dim_out = 1 #output dimension
dim_hid = 5 #initial hidden layer dimension

dim_C = 3 #fixed cascading layer dimension

varepochs = 100 #max number of epochs before adding casc layer (100 in technique paper)
batch = 5 #batch size

### Constructive Cascading NN

In [6]:
class CCNNet(torch.nn.Module):
    
    def __init__(self, dim_in, dim_out, hid_lay=False):
        super(N_Layer_Net, self).__init__()
        self.dim_in = dim_in #input dimension
        self.dim_hid = dim_hid #hidden layer dimension
        self.dim_out = dim_out #output dimension
        self.hid_lay = hid_lay
        
        self.hidl = None
        self.outl = None
        
        if hid_lay == True:
            self.hidl = torch.nn.Linear(dim_in, dim_hid) #initial hidden layer
            self.outl = torch.nn.Linear(dim_hid, dim_out)
        else:
            self.outl = torch.nn.Linear(dim_in, dim_out)
        
        #Cascading Layers
        self.casc_lays = torch.nn.ModuleList([]) #store cascading layers
        self.cascl_size = [] #dimension of each cascading layer
    
    def forward(self, x):
        
        out = None
        if hid_lay == True:
            out = self.hidl(x)
        
        out = self.hidl(x)
        out = F.tanh(out)
        out = self.outl(out)
        
#         layer_in = np.empty(shape = self.nhidlays,
#                            dtype = torch.Tensor)
#         layer_out = np.empty(shape = self.nhidlays,
#                              dtype = torch.Tensor)
        
#         for i in range(self.hid_lays.shape[0]):
#             if i == 0:
#                 layer_out[i] = x
#             else:
#                 layer_in[i] = F.sigmoid(layer_out[i-1])
#                 layer_out[i] = self.hid_lays(layer_in[i])
        
#         hid_layer_output = self.hid_lays[-1](x)
#         out_layer_input = F.sigmoid(hid_layer_output)
#         out_layer_output = self.out_layer(out_layer_input)

        return out_layer_output
    
    
    def addCascLayer(self, size=1):
        """
        add cascading layer to NN
        """
        ind = 0 #store input dimension of new casc layer
        outd = self.dim_out #output dimension of new casc layer
        
        if self.hid_lay == True:
            ind = self.dim_in + self.dim_hid
        else:
            ind = self.dim_in
        ind += sum(self.cascl_size)
        
        self.cascl_size = np.append(self.cascl_size, size) #add size of new casc layer to list
        self.casc_lays.append(torch.nn.Linear(ind, outd)) #add casc layer to ModuleList
        
        return None


### Pytorch Docs:
* nn.ModuleList: https://pytorch.org/docs/stable/nn.html#modulelist